In [1]:
%%time

import pathlib
from os.path import expanduser, join, abspath

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as fn
from pyspark.sql.types import *
from pyspark.sql.window import Window

warehouse_location = abspath('/home/jovyan/work/hive-db/spark-warehouse')
conf_jar = ("spark.jars", "scala//target/scala-2.11/sparkudfs_2.11-0.1.jar")

spark = SparkSession \
        .builder \
        .config("spark.sql.warehouse.dir", warehouse_location) \
        .config(*conf_jar) \
        .enableHiveSupport() \
        .appName("local-test") \
        .getOrCreate()

spark

CPU times: user 486 ms, sys: 224 ms, total: 710 ms
Wall time: 6.32 s


- configの`spark.jars`でjarファイルを渡せる
- https://spark.apache.org/docs/latest/configuration.html

In [3]:
spark.udf.registerJavaFunction("add_one", "com.example.spark.udfs.addOne","integer")

In [5]:
%%time
# prepare test-data
import seaborn as sns
df_flights = spark.createDataFrame(sns.load_dataset('flights'))
df_flights.createOrReplaceTempView('flights')
spark.sql('select * from flights').show()

/usr/local/spark/python/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion from Arrow: dictionary<values=string, indices=int8, ordered=0>
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


+----+---------+----------+
|year|    month|passengers|
+----+---------+----------+
|1949|  January|       112|
|1949| February|       118|
|1949|    March|       132|
|1949|    April|       129|
|1949|      May|       121|
|1949|     June|       135|
|1949|     July|       148|
|1949|   August|       148|
|1949|September|       136|
|1949|  October|       119|
|1949| November|       104|
|1949| December|       118|
|1950|  January|       115|
|1950| February|       126|
|1950|    March|       141|
|1950|    April|       135|
|1950|      May|       125|
|1950|     June|       149|
|1950|     July|       170|
|1950|   August|       170|
+----+---------+----------+
only showing top 20 rows

CPU times: user 957 ms, sys: 179 ms, total: 1.14 s
Wall time: 14.7 s


In [6]:
%%time

# test scala-udf

spark.sql("""
select *, add_one(passengers) as added from flights
""").show()

Py4JJavaError: An error occurred while calling o134.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$259: (bigint) => int)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:255)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:247)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$24.apply(RDD.scala:836)
	at org.apache.spark.rdd.RDD$$anonfun$mapP

In [7]:
%%time

# test scala-udf

spark.sql("""
select *, add_one(cast(passengers as int)) as added from flights
""").show()

+----+---------+----------+-----+
|year|    month|passengers|added|
+----+---------+----------+-----+
|1949|  January|       112|  113|
|1949| February|       118|  119|
|1949|    March|       132|  133|
|1949|    April|       129|  130|
|1949|      May|       121|  122|
|1949|     June|       135|  136|
|1949|     July|       148|  149|
|1949|   August|       148|  149|
|1949|September|       136|  137|
|1949|  October|       119|  120|
|1949| November|       104|  105|
|1949| December|       118|  119|
|1950|  January|       115|  116|
|1950| February|       126|  127|
|1950|    March|       141|  142|
|1950|    April|       135|  136|
|1950|      May|       125|  126|
|1950|     June|       149|  150|
|1950|     July|       170|  171|
|1950|   August|       170|  171|
+----+---------+----------+-----+
only showing top 20 rows

CPU times: user 2.75 ms, sys: 0 ns, total: 2.75 ms
Wall time: 316 ms


- cast(value as int)すると通る

- 参考文献：
    - https://qiita.com/neppysan/items/f90156635571a0c327c6

- DataFrame-APIで使える関数の作成
    - 上記のscalaクラスのjarファイルを使って試みたが失敗している
    - https://stackoverflow.com/questions/36171208/implement-a-java-udf-and-call-it-from-pyspark
    - http://ja.voidcc.com/question/p-euycvicn-bq.html

In [9]:
def my_udf(col): 
    from pyspark.sql.column import Column, _to_java_column, _to_seq 
    pcls = "com.example.spark.udfs.addOne" 
    jc = sc._jvm.java.lang.Thread.currentThread() \
     .getContextClassLoader().loadClass(pcls).newInstance().getUdf().apply 
    return Column(jc(_to_seq(sc, [col], _to_java_column))) 

In [10]:
my_udf

<function __main__.my_udf(col)>

In [11]:
df_flights.show()

+----+---------+----------+
|year|    month|passengers|
+----+---------+----------+
|1949|  January|       112|
|1949| February|       118|
|1949|    March|       132|
|1949|    April|       129|
|1949|      May|       121|
|1949|     June|       135|
|1949|     July|       148|
|1949|   August|       148|
|1949|September|       136|
|1949|  October|       119|
|1949| November|       104|
|1949| December|       118|
|1950|  January|       115|
|1950| February|       126|
|1950|    March|       141|
|1950|    April|       135|
|1950|      May|       125|
|1950|     June|       149|
|1950|     July|       170|
|1950|   August|       170|
+----+---------+----------+
only showing top 20 rows



In [12]:
df_flights.dtypes

[('year', 'bigint'), ('month', 'string'), ('passengers', 'bigint')]

In [13]:
df_flights.withColumn("added", my_udf("passengers")).show()

NameError: name 'sc' is not defined

In [17]:
sc = spark.sparkContext
def my_udf(col): 
    from pyspark.sql.column import Column, _to_java_column, _to_seq 
    pcls = "com.example.spark.udfs.addOne" 
    jc = sc._jvm.java.lang.Thread.currentThread() \
     .getContextClassLoader().loadClass(pcls).newInstance().getUdf().apply 
    return Column(jc(_to_seq(sc, [col], _to_java_column))) 

In [18]:
my_udf

<function __main__.my_udf(col)>

In [19]:
df_flights.withColumn("added", my_udf("passengers")).show()

Py4JError: An error occurred while calling o188.getUdf. Trace:
py4j.Py4JException: Method getUdf([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)



- DataFrame-APIの中で使えるような関数・集約関数の形にするのは若干手間がかかりそう